In [6]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
roberta = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")

# 새로운 문장 예시
org_text = "마을골목길에퇴비를두어 악 취 발생이 심 하니 이동 조치해 주세요.".replace(" ", "")  # 공백 제거
label = ["UNK", "PAD", "O", "B", "I", "E", "S"] 
"""
UNK: 알 수 없는 토큰에 대한 태그
PAD: 패딩에 해당하는 태그(문장의 길이를 맞추기 위해 사용)
O: 띄어쓰기가 필요하지 않은 문자
B: 새로운 단어의 시작 부분
I: 단어의 중간 부분
E: 단어의 끝 부분 - 해당 태그가 있으면 띄어쓰기가 필요함
S: 단일 문자로 구성된 단어
"""

# char 단위로 토큰화
token_list = [tokenizer.cls_token_id]
for char in org_text:
    token_list.append(tokenizer.encode(char)[1]) 
token_list.append(tokenizer.eos_token_id)
tkd = torch.tensor(token_list).unsqueeze(0)

output = roberta(tkd).logits

_, pred_idx = torch.max(output, dim=2)
tags = [label[idx] for idx in pred_idx.squeeze()][1:-1]
pred_sent = ""
for char_idx, spc_idx in enumerate(pred_idx.squeeze()[1:-1]):
    # "E" 태그를 기준으로 띄어쓰기
    if label[spc_idx] == "E": 
        pred_sent += org_text[char_idx] + " "
    else: 
        pred_sent += org_text[char_idx]

print(pred_sent.strip())


마을 골목길에 퇴비를 두어 악취 발생이 심하니 이동조치 해주세요.


## 성능 테스트

### 변환 후 전처리

In [1]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")

# GPU 사용 가능시 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 레이블 정의
label = ["UNK", "PAD", "O", "B", "I", "E", "S"]

def apply_spacing_correction(text):
    if not isinstance(text, str):
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 512 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 512 토큰 단위로 나누기
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            token_list = [tokenizer.cls_token_id]
            for char in chunk:
                token_list.extend(tokenizer.encode(char)[1:-1])  # CLS, SEP 토큰 제외
            token_list.append(tokenizer.sep_token_id)
            
            tkd = torch.tensor(token_list).unsqueeze(0).to(device)
            
            # 모델 예측
            with torch.no_grad():
                outputs = model(tkd)
                pred_idx = torch.argmax(outputs.logits, dim=-1)
            
            # 예측 결과를 문자열로 변환
            pred_sent = ""
            for char_idx, spc_idx in enumerate(pred_idx.squeeze()[1:-1]):
                if char_idx >= len(chunk):  # 인덱스 범위 체크
                    break
                
                curr_label = label[spc_idx]
                if curr_label in ["E", "S"]:  # E나 S 태그가 있으면 띄어쓰기 추가
                    pred_sent += chunk[char_idx] + " "
                else:
                    pred_sent += chunk[char_idx]
            
            corrected_sentences.append(pred_sent.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip()
    
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return text  # 에러 발생시 원본 텍스트 반환

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 띄어쓰기 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(apply_spacing_correction)
    
    return df


/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-13 16:43:28.800008: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 16:43:28.826413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 16:43:29.167869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [2]:
import json
import pandas as pd


with open('/home/yjtech2/Desktop/yurim/LLM/DATA/띄어쓰기문장부호오류.json', 'r') as f:
    json_data = json.load(f)
    
data = []
for item in json_data['data']:
    if 'annotation' in item:
        annotation = item['annotation']
        
        if 'err_sentence' in annotation and 'cor_sentence' in annotation:
            data.append({
                'err_sentence': annotation['err_sentence'],
                'cor_sentence': annotation['cor_sentence']
            })
            
df = pd.DataFrame(data)

if __name__ == "__main__":        
    # 띄어쓰기 교정 처리
    processed_df = process_dataframe(df, "err_sentence")
    print('처리 완료!')

Processing spacing correction: 100%|██████████| 36000/36000 [01:50<00:00, 324.73it/s]

처리 완료!


In [3]:
import pandas as pd
import re
from difflib import SequenceMatcher

def add_comparison_columns(df):
    
    def calculate_spacing_similarity(row):
        """
        띄어쓰기를 포함한 두 문장 간의 유사도를 계산하는 함수
        """
        if pd.isna(row['res_sentence']) or pd.isna(row['cor_sentence']):
            return 0
        
        # 문장의 띄어쓰기를 유지한 채로 비교
        res = row['res_sentence']
        cor = row['cor_sentence']
        
        # 완전히 같으면 1 반환
        if res == cor:
            return 1.0
        
        # 다르면 띄어쓰기를 포함한 유사도 계산
        similarity = SequenceMatcher(None, res, cor).ratio()
        
        # 띄어쓰기 패턴의 정확도를 별도로 계산
        res_spaces = [i for i, char in enumerate(res) if char == ' ']
        cor_spaces = [i for i, char in enumerate(cor) if char == ' ']
        
        # 공통된 띄어쓰기 위치 수 계산
        common_spaces = len(set(res_spaces) & set(cor_spaces))
        total_spaces = len(set(res_spaces) | set(cor_spaces))
        
        # 띄어쓰기 정확도 (공통 띄어쓰기 / 전체 띄어쓰기)
        spacing_accuracy = common_spaces / total_spaces if total_spaces > 0 else 1.0
        
        # 전체 유사도와 띄어쓰기 정확도를 조합 (가중치는 조정 가능)
        final_ratio = (similarity + spacing_accuracy) / 2
        
        return round(final_ratio, 3)
    
    # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
    df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
    
    # ratio 컬럼 추가 (띄어쓰기를 고려한 유사도 계산)
    df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
    
    return df

# 사용 예시
if __name__ == "__main__":
    processed_df = processed_df[['err_sentence', 'res_sentence', 'cor_sentence']]
    
    def text_clean(text):
        # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        # text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    
    result_df = add_comparison_columns(processed_df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))

비교 완료
정확도:  0.7749166666666667
비율 평균:  0.916460055555547


In [55]:
result_df

,err_sentence,res_sentence,cor_sentence,check,ratio
0,일손과시간이부족해 검증을 다못했다는 군색한변명은안통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안 통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안 통한다,1,1.0
1,총리 후보가사퇴할때마다대통령에게 오는타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나다,1,1.0
2,이정부 들어 처음으로 박 대통령에대한부정적평가가 긍정적평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이다,1,1.0
3,대통령이지지율에일희일비할필요는 없다.,대통령이 지지율에 일희일비할 필요는 없다,대통령이 지지율에 일희일비할 필요는 없다,1,1.0
4,위대한 업적을 남긴지도자일수록 반대여론을 무릅쓴 경우가많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다,1,1.0
...,...,...,...,...,...
35995,서울 벗어나면 놀이터가없는마을도,서울 벗어나면 놀이터가 없는 마을도,서울 벗어나면 놀이터가 없는 마을도,1,1.0
35996,집 밖으로나가 뛰어놀고싶은 어린이들에게 놀이터는 너무 멀리있거나 즐겁게 놀 만한기구...,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...,1,1.0
35997,재미가없으니 어린이들이모이지않고 같이 놀 친구가없어서 더욱 갈일이없어진다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진다,1,1.0
35998,어린이들이 뛰어놀수있는 공간마저도 지역에따라 격차를 보이고있는것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이다,1,1.0


In [4]:
result_df[result_df['check'] == 0]

,err_sentence,res_sentence,cor_sentence,check,ratio
5,하지만 국민 지지가 뒷받침되면 국정 운영에 자신감과 탄력이더해진다는점도 분명하다,하지만 국민지지가 뒷받침되면 국정운영에 자신감과 탄력이 더해진다는 점도 분명하다,하지만 국민 지지가 뒷받침되면 국정 운영에 자신감과 탄력이 더해진다는 점도 분명하다,0,0.518
11,국민 혈세로시장을 틀어막아 달라는것은 어불성설이다.,국민혈세로 시장을 틀어막아 달라는 것은 어불성설이다,국민 혈세로 시장을 틀어막아 달라는 것은 어불성설이다,0,0.491
12,차제에문제 병사 관리 실태를 일제히 점검하라,차제에 문제병사 관리 실태를 일제히 점검하라,차제에 문제 병사 관리 실태를 일제히 점검하라,0,0.540
13,당시에도 군 경계 태세에 구멍이 뚫렸다는 비판이들끓었는데 또 이사단에서 사고가 터졌다.,당시에도 군경계 태세에 구멍이 뚫렸다는 비판이 들끓었는데 또 이 사단에서 사고가 터졌다,당시에도 군 경계 태세에 구멍이 뚫렸다는 비판이 들끓었는데 또 이 사단에서 사고가 터졌다,0,0.518
14,군 총기사고는 잊을만하면발생한다.,군총기 사고는 잊을 만하면 발생한다,군 총기 사고는 잊을 만하면 발생한다,0,0.487
...,...,...,...,...,...
35964,그러나 딸은죽은 엄마와의추억이 떠올라이미 오래전부터 피아노 레슨을 받지않고있었다.,그러나 딸은 죽은 엄마와의 추억이 떠올라 이미 오래 전부터 피아노 레슨을 받지 않고...,그러나 딸은 죽은 엄마와의 추억이 떠올라 이미 오래전부터 피아노 레슨을 받지 않고 있었다,0,0.689
35970,"저출산, 저소득층의 사고, 장병의 불확실한 미래는기업의지속가능성을위협하는사회적 문제다.",저출산 저소득층의 사고 장병의 불확실한 미래는 기업의 지속가능성을 위협하는 사회적 문제다,저출산 저소득층의 사고 장병의 불확실한 미래는 기업의 지속 가능성을 위협하는 사회적...,0,0.745
35981,경선에선 역대 최대여성후보 바람,경선에 선 역대 최대 여성 후보 바람,경선에선 역대 최대 여성 후보 바람,0,0.487
35986,최대 관심사는 네이버가 유럽에서 첫선을보일 서비스다,최대 관심사는 네이버가 유럽에서 첫 선을 보일 서비스다,최대 관심사는 네이버가 유럽에서 첫선을 보일 서비스다,0,0.714


In [5]:
result_df.to_excel('./hugging_result.xlsx', index = False)

### 전처리 후 변환

In [5]:
import json
import pandas as pd


with open('/home/yjtech2/Desktop/yurim/LLM/DATA/띄어쓰기문장부호오류.json', 'r') as f:
    json_data = json.load(f)
    
data = []
for item in json_data['data']:
    if 'annotation' in item:
        annotation = item['annotation']
        
        if 'err_sentence' in annotation and 'cor_sentence' in annotation:
            data.append({
                'err_sentence': annotation['err_sentence'],
                'cor_sentence': annotation['cor_sentence']
            })
            
df = pd.DataFrame(data)

def text_clean(text):
    # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    # text = re.sub(pattern, '', text)
    
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)

    return text 

df['err_sentence'] = df['err_sentence'].apply(text_clean)
df['cor_sentence'] = df['cor_sentence'].apply(text_clean)

In [ ]:
df

,err_sentence,cor_sentence
0,일손과시간이부족해 검증을 다못했다는 군색한변명은안통한다,일손과 시간이 부족해 검증을 다 못했다는 군색한 변명은 안 통한다
1,총리 후보가사퇴할때마다대통령에게 오는타격은 엄청나다,총리 후보가 사퇴할 때마다 대통령에게 오는 타격은 엄청나다
2,이정부 들어 처음으로 박 대통령에대한부정적평가가 긍정적평가를 넘어선 것이다,이 정부 들어 처음으로 박 대통령에 대한 부정적 평가가 긍정적 평가를 넘어선 것이다
3,대통령이지지율에일희일비할필요는 없다,대통령이 지지율에 일희일비할 필요는 없다
4,위대한 업적을 남긴지도자일수록 반대여론을 무릅쓴 경우가많다,위대한 업적을 남긴 지도자일수록 반대 여론을 무릅쓴 경우가 많다
...,...,...
35995,서울 벗어나면 놀이터가없는마을도,서울 벗어나면 놀이터가 없는 마을도
35996,집 밖으로나가 뛰어놀고싶은 어린이들에게 놀이터는 너무 멀리있거나 즐겁게 놀 만한기구가없다,집 밖으로 나가 뛰어놀고 싶은 어린이들에게 놀이터는 너무 멀리 있거나 즐겁게 놀 만...
35997,재미가없으니 어린이들이모이지않고 같이 놀 친구가없어서 더욱 갈일이없어진다,재미가 없으니 어린이들이 모이지 않고 같이 놀 친구가 없어서 더욱 갈 일이 없어진다
35998,어린이들이 뛰어놀수있는 공간마저도 지역에따라 격차를 보이고있는것이다,어린이들이 뛰어놀 수 있는 공간마저도 지역에 따라 격차를 보이고 있는 것이다


In [8]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")

# GPU 사용 가능시 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 레이블 정의
label = ["UNK", "PAD", "O", "B", "I", "E", "S"]

def apply_spacing_correction(text):
    if not isinstance(text, str):
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 512 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 512 토큰 단위로 나누기
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            token_list = [tokenizer.cls_token_id]
            for char in chunk:
                token_list.extend(tokenizer.encode(char)[1:-1])  # CLS, SEP 토큰 제외
            token_list.append(tokenizer.sep_token_id)
            
            tkd = torch.tensor(token_list).unsqueeze(0).to(device)
            
            # 모델 예측
            with torch.no_grad():
                outputs = model(tkd)
                pred_idx = torch.argmax(outputs.logits, dim=-1)
            
            # 예측 결과를 문자열로 변환
            pred_sent = ""
            for char_idx, spc_idx in enumerate(pred_idx.squeeze()[1:-1]):
                if char_idx >= len(chunk):  # 인덱스 범위 체크
                    break
                
                curr_label = label[spc_idx]
                if curr_label in ["E", "S"]:  # E나 S 태그가 있으면 띄어쓰기 추가
                    pred_sent += chunk[char_idx] + " "
                else:
                    pred_sent += chunk[char_idx]
            
            corrected_sentences.append(pred_sent.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip()
    
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return text  # 에러 발생시 원본 텍스트 반환

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 띄어쓰기 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(apply_spacing_correction)
    
    return df

if __name__ == "__main__":        
    # 띄어쓰기 교정 처리
    processed_df = process_dataframe(df, "err_sentence")
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    
    result_df = add_comparison_columns(processed_df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))
    print('처리 완료!')

Processing spacing correction: 100%|██████████| 36000/36000 [01:49<00:00, 329.82it/s]


비교 완료
정확도:  0.765
비율 평균:  0.9126824166666568
처리 완료!


In [1]:
import json
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm
import pandas as pd
import re
from difflib import SequenceMatcher
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기
def load_data(file_path):
    df = pd.read_csv(file_path, encoding = 'cp949')
    
    print(df.info())
    
    return df


# 모델 정의
tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")

# GPU 사용 가능시 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 레이블 정의
label = ["UNK", "PAD", "O", "B", "I", "E", "S"]

def apply_spacing_correction(text):
    if not isinstance(text, str):
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 512 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 512 토큰 단위로 나누기
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            token_list = [tokenizer.cls_token_id]
            for char in chunk:
                token_list.extend(tokenizer.encode(char)[1:-1])  # CLS, SEP 토큰 제외
            token_list.append(tokenizer.sep_token_id)
            
            tkd = torch.tensor(token_list).unsqueeze(0).to(device)
            
            # 모델 예측
            with torch.no_grad():
                outputs = model(tkd)
                pred_idx = torch.argmax(outputs.logits, dim=-1)
            
            # 예측 결과를 문자열로 변환
            pred_sent = ""
            for char_idx, spc_idx in enumerate(pred_idx.squeeze()[1:-1]):
                if char_idx >= len(chunk):  # 인덱스 범위 체크
                    break
                
                curr_label = label[spc_idx]
                if curr_label in ["E", "S"]:  # E나 S 태그가 있으면 띄어쓰기 추가
                    pred_sent += chunk[char_idx] + " "
                else:
                    pred_sent += chunk[char_idx]
            
            corrected_sentences.append(pred_sent.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip()
    
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return text  # 에러 발생시 원본 텍스트 반환

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 띄어쓰기 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(apply_spacing_correction)
    
    return df



# 수행
if __name__ == "__main__":
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Start ======')
    df = load_data('/home/yjtech2/Desktop/yurim/LLM/DATA/냄새_악취_포함.csv')
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Finished ======')
    
    print(f'[{_now_time}] ====== Data Preprocessing Start ======')
    processed_df = process_dataframe(df, "제목")
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Preprocessing Finished ======')

    # processed_df = processed_df[['err_sentence', 'res_sentence', 'cor_sentence']]
    
    def text_clean(text):
        # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        # text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    # processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    processed_df


/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-14 16:19:45.347088: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 16:19:45.372500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 16:19:45.712210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

[2024-11-14 16:19:47.702946] ====== Data Load Start ======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      116 non-null    object
 1   질문내용    116 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB
None
[2024-11-14 16:19:47.707436] ====== Data Load Finished ======
[2024-11-14 16:19:47.707436] ====== Data Preprocessing Start ======


Processing spacing correction: 100%|██████████| 116/116 [00:00<00:00, 291.63it/s]

[2024-11-14 16:19:48.107337] ====== Data Preprocessing Finished ======


In [2]:
processed_df

,제목,질문내용,res_sentence
0,악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다.,순창군 순창읍 남계리ㅇㅇㅇ일원\n악취 나는 물 이 계속 흘러 내려 물을 오염시키고 ...,악취 나는 뮬이 계속 흘러내려 물을 오염시키고 있습니다
1,마을 골 목 길에 퇴비를 두어 악취 발생이 심하니 이동 조치해 주세요.,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...,마을 골목길에 퇴비를 두어 악취 발생이 심하니 이동조치 해주세요
2,노후된 모정 보수 요청ㅓ,전라북도순창군구림면금천리89-2\n치천마을의치천옆에모정이있는데설치한지30년이넘어서나...,노후된 모정 보수 요청ㅓ
3,인도에 패기물 컨테이너 적치로 통행 불편,순창군 순창읍 ㅇㅇ리 ㅇㅇㅇ\n인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 ...,인도에 패기물 컨테이너 적치로 통행 불편
4,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,첨부 한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유...,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다
...,...,...,...
111,복합악취의 수인한도는?,복합악취의 수인한도는?,복합악취의 수인 한도는
112,악취배출량 단위 및 산정방법은?,악취배출량 단위 및 산정방법,악취 배출량 단위 및 산정 방법은
113,단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁조정대...,"단독 또는 공동주택에서 애완견이나 고양이 등 동물사육으로 인하여 소음, 악취 등을 ...",단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁 조정...
114,"악취배출사업장 기술지원 및 공공환경시설 악취기술진단, 악취물질 측정·분석 관련 질의응답",Q1: ○ (악취배출사업장 기술지원) 악취저감기술 지원의 신청방법과 무상지원 범위는...,악취배출사업장 기술지원 및 공공환경시설 악취기술 진단 악취물질 측정분석 관련 질의 응답


In [3]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
from tqdm import tqdm
import pandas as pd
import re
from difflib import SequenceMatcher

import warnings
warnings.filterwarnings('ignore')


def apply_spelling_correction(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 128 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 입력 텍스트가 비어있는 경우 처리
        if not org_text:
            return ""
        
        # 청크 단위로 처리
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            
            # 각 청크에 대해 모델 처리
            input_encoding = tokenizer("맞춤법을 고쳐주세요: " + chunk, return_tensors="pt")
            input_ids = input_encoding.input_ids.to(device)
            attention_mask = input_encoding.attention_mask.to(device)
            
            output_encoding = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=5,
                early_stopping=True,
            )
            
            chunk_result = tokenizer.decode(output_encoding[0], skip_special_tokens=True)
            if chunk_result.strip():  # 빈 문자열이 아닌 경우에만 추가
                corrected_sentences.append(chunk_result.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip() or ""  # 빈 문자열이면 "" 반환
        
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return ""

def process_dataframe(df, text_column):
    tqdm.pandas(desc="Processing spelling correction")
    df["spacing_sentence"] = df[text_column].progress_apply(apply_spelling_correction)

    
    return df



# 모델 정의 (전역 변수로 유지)
model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

if __name__ == "__main__":
    processed_df = process_dataframe(processed_df, "res_sentence")
    print('처리 완료!')

Processing spelling correction: 100%|██████████| 116/116 [00:07<00:00, 15.52it/s]

처리 완료!


In [4]:
processed_df

,제목,질문내용,res_sentence,spacing_sentence
0,악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다.,순창군 순창읍 남계리ㅇㅇㅇ일원\n악취 나는 물 이 계속 흘러 내려 물을 오염시키고 ...,악취 나는 뮬이 계속 흘러내려 물을 오염시키고 있습니다,냄새나는 뮬이 계속 흘러내려 물을 오염시키고 있습니다.
1,마을 골 목 길에 퇴비를 두어 악취 발생이 심하니 이동 조치해 주세요.,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...,마을 골목길에 퇴비를 두어 악취 발생이 심하니 이동조치 해주세요,마을 골목길에 퇴비를 두어 악취 발생이 심하니 이동 조치 해 주세요.
2,노후된 모정 보수 요청ㅓ,전라북도순창군구림면금천리89-2\n치천마을의치천옆에모정이있는데설치한지30년이넘어서나...,노후된 모정 보수 요청ㅓ,노후된 이모 정보 수 요청.
3,인도에 패기물 컨테이너 적치로 통행 불편,순창군 순창읍 ㅇㅇ리 ㅇㅇㅇ\n인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 ...,인도에 패기물 컨테이너 적치로 통행 불편,인도에 패기물 컨테이너 적치로통행 불편.
4,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,첨부 한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유...,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추측되는 물질이다량으로 유출되어 주변에 피해를 쥽니다.
...,...,...,...,...
111,복합악취의 수인한도는?,복합악취의 수인한도는?,복합악취의 수인 한도는,복합 악취의 수인 한도는.
112,악취배출량 단위 및 산정방법은?,악취배출량 단위 및 산정방법,악취 배출량 단위 및 산정 방법은,냄새 배출량 단위 및 산정 방법은.
113,단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁조정대...,"단독 또는 공동주택에서 애완견이나 고양이 등 동물사육으로 인하여 소음, 악취 등을 ...",단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁 조정...,단독 또는 공동주택에서 애완견을 사육하여 소음 등을 유발하는 경우 환경분쟁 조정 대...
114,"악취배출사업장 기술지원 및 공공환경시설 악취기술진단, 악취물질 측정·분석 관련 질의응답",Q1: ○ (악취배출사업장 기술지원) 악취저감기술 지원의 신청방법과 무상지원 범위는...,악취배출사업장 기술지원 및 공공환경시설 악취기술 진단 악취물질 측정분석 관련 질의 응답,냄새 배출 사업장 기술 지원 및 공공 환경 시설 악취 기술 진단 악취 물질 측정 분...
